# **FLO RFM Analizi**
## **İş Problemi**
Online ayakkabı mağazası olan FLO müşterilerini segmentlere ayırıp bu segmentlere göre pazarlama stratejileri belirlemek istiyor. Buna yönelik olarak müşterilerin davranışları tanımlanacak ve bu davranışlardaki öbeklenmelere göre gruplar oluşturulacak.

## **Veri Seti Hikayesi**
Veri seti Flo’dan son alışverişlerini 2020 - 2021 yıllarında OmniChannel (hem online hem offline alışveriş yapan) olarak yapan müşterilerin geçmiş alışveriş davranışlarından elde edilen bilgilerden oluşmaktadır.


### **DEGİSKENLER**
**master_id :** Eşsiz müşteri numarası

**order_channel :** Alışveriş yapılan platforma ait hangi kanalın kullanıldığı (Android, ios, Desktop, Mobile)

**last_order_channel:** En son alışverişin yapıldığı kanal

**first_order_date :** Müşterinin yaptığı ilk alışveriş tarihi

**last_order_date :** Müşterinin yaptığı son alışveriş tarihi

**last_order_date_online :** Müşterinin online platformda yaptığı son alışveriş tarihi

**last_order_date_offline :** Müşterinin offline platformda yaptığı son alışveriş tarihi

**order_num_total_ever_online :** Müşterinin online platformda yaptığı toplam alışveriş sayısı

**order_num_total_ever_offline :** Müşterinin offline'da yaptığı toplam alışveriş sayısı

**customer_value_total_ever_offline :** Müşterinin offline alışverişlerinde ödediği toplam ücret

**customer_value_total_ever_online :** Müşterinin online alışverişlerinde ödediği toplam ücret

**interested_in_categories_12 :** Müşterinin son 12 ayda alışveriş yaptığı kategorilerin listesi

#### **Görev 1**: Veriyi anlama ve hazırlama
##### **Adım 1:** flo_data_20K.csv verisini okuyunuz. Dataframe'in kopyasını oluşturunuz.

In [1]:
from google.colab import drive
drive.mount("/content/gdrive")

Drive already mounted at /content/gdrive; to attempt to forcibly remount, call drive.mount("/content/gdrive", force_remount=True).


In [2]:
!pip install lifetimes 

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [3]:
import pandas as pd
import datetime as dt
from lifetimes import BetaGeoFitter
from lifetimes import GammaGammaFitter
from sklearn.preprocessing import MinMaxScaler
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)
pd.set_option('display.float_format', lambda x: '%.2f' % x)
pd.options.mode.chained_assignment = None

In [4]:
df_ = pd.read_csv("/content/gdrive/MyDrive/DSMLBC10/week_4 (20.10.22-26.10.22)/projects/flo_data_20k.csv")
df= df_.copy()
df.head()

,master_id,order_channel,last_order_channel,first_order_date,last_order_date,last_order_date_online,last_order_date_offline,order_num_total_ever_online,order_num_total_ever_offline,customer_value_total_ever_offline,customer_value_total_ever_online,interested_in_categories_12
0,cc294636-19f0-11eb-8d74-000d3a38a36f,Android App,Offline,2020-10-30,2021-02-26,2021-02-21,2021-02-26,4.00,1.00,139.99,799.38,[KADIN]
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,Android App,Mobile,2017-02-08,2021-02-16,2021-02-16,2020-01-10,19.00,2.00,159.97,1853.58,"[ERKEK, COCUK, KADIN, AKTIFSPOR]"
2,69b69676-1a40-11ea-941b-000d3a38a36f,Android App,Android App,2019-11-27,2020-11-27,2020-11-27,2019-12-01,3.00,2.00,189.97,395.35,"[ERKEK, KADIN]"
3,1854e56c-491f-11eb-806e-000d3a38a36f,Android App,Android App,2021-01-06,2021-01-17,2021-01-17,2021-01-06,1.00,1.00,39.99,81.98,"[AKTIFCOCUK, COCUK]"
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,Desktop,Desktop,2019-08-03,2021-03-07,2021-03-07,2019-08-03,1.00,1.00,49.99,159.99,[AKTIFSPOR]


##### **Adım 2:**  Aykırı değerleri baskılamak için gerekli olan outlier_thresholds ve replace_with_thresholds fonksiyonlarını tanımlayınız.
**Not:** cltv hesaplanırken frequency değerleri integer olması gerekmektedir.Bu nedenle alt ve üst limitlerini round() ile yuvarlayınız.


In [5]:
def outlier_thresholds(dataframe, variable):
    quartile1 = dataframe[variable].quantile(0.01)
    quartile3 = dataframe[variable].quantile(0.99)
    interquantile_range = quartile3 - quartile1
    up_limit = quartile3 + 1.5 * interquantile_range
    low_limit = quartile1 - 1.5 * interquantile_range
    return low_limit, up_limit

def replace_with_thresholds(dataframe, variable):
    low_limit, up_limit = outlier_thresholds(dataframe, variable)
    # dataframe.loc[(dataframe[variable] < low_limit), variable] = low_limit
    dataframe.loc[(dataframe[variable] > up_limit), variable] = up_limit

##### **Adım 3:**  "order_num_total_ever_online","order_num_total_ever_offline","customer_value_total_ever_offline","customer_value_total_ever_online" değişkenlerinin aykırı değerleri varsa baskılayınız.

In [6]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.11,4.23,1.00,1.00,2.00,4.00,200.00
order_num_total_ever_offline,19945.00,1.91,2.06,1.00,1.00,1.00,2.00,109.00
customer_value_total_ever_offline,19945.00,253.92,301.53,10.00,99.99,179.98,319.97,18119.14
customer_value_total_ever_online,19945.00,497.32,832.60,12.99,149.98,286.46,578.44,45220.13


In [7]:
replace_with_thresholds(df, "order_num_total_ever_online")
replace_with_thresholds(df, "order_num_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_offline")
replace_with_thresholds(df, "customer_value_total_ever_online")

In [8]:
df.describe().T

,count,mean,std,min,25%,50%,75%,max
order_num_total_ever_online,19945.00,3.09,3.81,1.00,1.00,2.00,4.00,48.50
order_num_total_ever_offline,19945.00,1.89,1.43,1.00,1.00,1.00,2.00,16.00
customer_value_total_ever_offline,19945.00,251.92,251.02,10.00,99.99,179.98,319.97,3019.88
customer_value_total_ever_online,19945.00,489.71,632.61,12.99,149.98,286.46,578.44,7799.54


##### **Adım 4:** Omnichannel müşterilerin hem online'dan hem de offline platformlardan alışveriş yaptığını ifade etmektedir. Her bir müşterinin toplam alışveriş sayısı ve harcaması için yeni değişkenler oluşturunuz.

In [9]:
df["order_num_total"] = df["order_num_total_ever_online"] + df["order_num_total_ever_offline"]
df["customer_value_total_price"] = df["customer_value_total_ever_offline"] + df["customer_value_total_ever_online"]

##### **Adım 5:** Değişken tiplerini inceleyiniz. Tarih ifade eden değişkenlerin tipini date'e çeviriniz.

In [10]:
df.dtypes

master_id                             object
order_channel                         object
last_order_channel                    object
first_order_date                      object
last_order_date                       object
last_order_date_online                object
last_order_date_offline               object
order_num_total_ever_online          float64
order_num_total_ever_offline         float64
customer_value_total_ever_offline    float64
customer_value_total_ever_online     float64
interested_in_categories_12           object
order_num_total                      float64
customer_value_total_price           float64
dtype: object

In [11]:
import datetime as dt

In [12]:
for col in df.columns:
  if "date" in col:
    df[col] = df[col].apply(pd.to_datetime)

In [13]:
df.dtypes

master_id                                    object
order_channel                                object
last_order_channel                           object
first_order_date                     datetime64[ns]
last_order_date                      datetime64[ns]
last_order_date_online               datetime64[ns]
last_order_date_offline              datetime64[ns]
order_num_total_ever_online                 float64
order_num_total_ever_offline                float64
customer_value_total_ever_offline           float64
customer_value_total_ever_online            float64
interested_in_categories_12                  object
order_num_total                             float64
customer_value_total_price                  float64
dtype: object

##### **Görev 2:** CLTV VERI YAPISININ OLUSTURULMASI
##### **Adım 1:** Veri setindeki en son alışverişin yapıldığı tarihten 2 gün sonrasını analiz tarihi olarak alınız.

In [14]:
df["last_order_date"].max()

Timestamp('2021-05-30 00:00:00')

In [15]:
analysis_date = dt.datetime(2021, 6, 1)
type(analysis_date)

datetime.datetime

##### **Adım 2:** customer_id, recency_cltv_weekly, T_weekly, frequency ve monetary_cltv_avg değerlerinin yer aldığı yeni bir cltv dataframe'i oluşturunuz. Monetary değeri satın alma başına ortalama değer olarak, recency ve tenure değerleri ise haftalık cinsten ifade edilecek.


In [16]:
#df["total_order"]=df["order_num_total_ever_online"]+df.order_num_total_ever_offline

#df["monetary"]=df.customer_value_total_ever_offline+df.customer_value_total_ever_online
#df.loc[:, df.columns.str.contains("date")]=df.loc[:, df.columns.str.contains("date")]\
                                             #.apply(pd.to_datetime,format='%Y-%m-%d')
#df.last_order_date.max()
#Timestamp('2021-05-30 00:00:00')
#today_date=dt.datetime(2021,6,2)
#df["T"]=(today_date-df.first_order_date).dt.days
#df["recency"]=(df.last_order_date-df.first_order_date).dt.days

In [17]:
cltv_df = pd.DataFrame()
cltv_df["customer_id"] = df["master_id"]
cltv_df["recency_cltv_weekly"] = ((df.last_order_date-df.first_order_date).dt.days) / 7
cltv_df["T_weekly"] = ((analysis_date - df["first_order_date"]).astype('timedelta64[D]'))/7
cltv_df["frequency"] = df["order_num_total"]
cltv_df["monetary_cltv_avg"] = df["customer_value_total_price"] / df["order_num_total"]
cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99


##### **Görev 3:** BG/NBD, Gamma-Gamma Modellerinin Kurulması ve CLTV’nin Hesaplanması

##### **Adım 1:** BG/NBD modelini fit ediniz.

3 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_3_month olarak cltv dataframe'ine ekleyiniz.

6 ay içerisinde müşterilerden beklenen satın almaları tahmin ediniz ve exp_sales_6_month olarak cltv dataframe'ine ekleyiniz.


In [18]:
bgf = BetaGeoFitter(penalizer_coef=0.001)
bgf.fit(cltv_df["frequency"],
        cltv_df["recency_cltv_weekly"],
        cltv_df["T_weekly"])

<lifetimes.BetaGeoFitter: fitted with 19945 subjects, a: 0.00, alpha: 76.17, b: 0.00, r: 3.66>

In [19]:
cltv_df["exp_sales_3_month"] = bgf.predict(4 * 3,
                                               cltv_df["frequency"],
                                               cltv_df["recency_cltv_weekly"],
                                               cltv_df["T_weekly"])

In [20]:
cltv_df["exp_sales_6_month"] = bgf.predict(4 * 6,
                                               cltv_df["frequency"],
                                               cltv_df["recency_cltv_weekly"],
                                               cltv_df["T_weekly"])

cltv_df.head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month
0,cc294636-19f0-11eb-8d74-000d3a38a36f,17.00,30.57,5.00,187.87,0.97,1.95
1,f431bd5a-ab7b-11e9-a2fc-000d3a38a36f,209.86,224.86,21.00,95.88,0.98,1.97
2,69b69676-1a40-11ea-941b-000d3a38a36f,52.29,78.86,5.00,117.06,0.67,1.34
3,1854e56c-491f-11eb-806e-000d3a38a36f,1.57,20.86,2.00,60.98,0.70,1.40
4,d6ea1074-f1f5-11e9-9346-000d3a38a36f,83.14,95.43,2.00,104.99,0.40,0.79


In [21]:
cltv_df["frequency"].astype(int).head()

0     5
1    21
2     5
3     2
4     2
Name: frequency, dtype: int64

##### **Adım 2:** Gamma-Gamma modelini fit ediniz. Müşterilerin ortalama bırakacakları değeri tahminleyip exp_average_value olarak cltv dataframe'ine ekleyiniz.



In [22]:
ggf = GammaGammaFitter(penalizer_coef=0.01)

ggf.fit(cltv_df["frequency"].astype(int), cltv_df["monetary_cltv_avg"])

<lifetimes.GammaGammaFitter: fitted with 19945 subjects, p: 4.15, q: 0.47, v: 4.08>

In [23]:
cltv_df["exp_average_value"] = ggf.conditional_expected_average_profit(cltv_df["frequency"],
                                        cltv_df["monetary_cltv_avg"])

In [24]:
cltv_df.sort_values("exp_average_value", ascending=False).head()

,customer_id,recency_cltv_weekly,T_weekly,frequency,monetary_cltv_avg,exp_sales_3_month,exp_sales_6_month,exp_average_value
9055,47a642fe-975b-11eb-8c2a-000d3a38a36f,2.86,7.86,4.00,1401.77,1.09,2.19,1449.03
17323,f59053e2-a503-11e9-a2fc-000d3a38a36f,51.71,101.00,7.00,1106.47,0.72,1.44,1127.61
15516,9083981a-f59e-11e9-841e-000d3a38a36f,63.57,83.86,4.00,1090.36,0.57,1.15,1127.35
6402,851de3b4-8f0c-11eb-8cb8-000d3a38a36f,8.29,9.43,2.00,862.69,0.79,1.59,923.68
16410,6fecd6c8-261a-11ea-8e1c-000d3a38a36f,57.00,94.86,2.00,859.58,0.40,0.79,920.36


##### **Adım 3:** 6 aylık CLTV hesaplayınız ve cltv ismiyle dataframe'e ekleyiniz.

 Cltv değeri en yüksek 20 kişiyi gözlemleyiniz.

In [25]:
cltv = ggf.customer_lifetime_value(bgf,
                                   cltv_df["frequency"],
                                   cltv_df["recency_cltv_weekly"],
                                   cltv_df["T_weekly"],
                                   cltv_df["monetary_cltv_avg"],
                                   time=6,  # 6 aylık / aylık olmasına dikkat et
                                   freq="W",  # T'nin frekans bilgisi.
                                   discount_rate=0.01)
cltv_df["cltv"] = cltv

In [26]:
cltv_df["cltv"].sort_values(ascending = False).head(20)

9055    3327.71
13880   3172.22
17323   1708.98
12438   1662.52
7330    1627.74
8868    1623.81
6402    1538.86
6666    1529.23
19538   1485.82
14858   1423.00
17963   1362.61
15516   1359.44
6717    1355.44
4157    1353.53
4735    1334.83
11694   1297.52
11179   1286.14
1853    1285.23
5775    1282.58
7312    1263.19
Name: cltv, dtype: float64

##### **Görev 4:** CLTV Değerine Göre Segmentlerin Oluşturulması

##### **Adım 1:** 6 aylık CLTV'ye göre tüm müşterilerinizi 4 gruba (segmente) ayırınız ve grup isimlerini veri setine ekleyiniz.


In [ ]:
cltv_df

In [ ]:
cltv_df["segment"] = pd.qcut(cltv_df["cltv"], 4, labels=["D", "C", "B", "A"])
cltv_df.head()

##### **Adım 2:** 4 grup içerisinden seçeceğiniz 2 grup için yönetime kısa kısa 6 aylık aksiyon önerilerinde bulununuz.

In [ ]:
cltv_df.groupby("segment").agg({"exp_sales_6_month": ["count", "mean", "sum","median"]})

A ve D segmenti özelinde konuşmak gerekirse, A segmenti, kısıtlı imkanlarım varsa ilgilenmem gereken bir gruptur firma için değerli müşterilerdir. D segmenti imkanlarım doğrultusunda daha çok ilgilenilmesi kampanya yapılması gereken gruptur.